# Calling OpenAI API to summarize a patent

First we get the patent details from Google and use it to prepare a prompt for calling the API. Then we build a user interface around the function.

In [29]:
# imports
from bs4 import BeautifulSoup
import requests
from openai import OpenAI
from dotenv import load_dotenv
import os
import gradio as gr
import google.generativeai as genai

In [30]:
# set the model

MODEL_GPT = 'gpt-4o-mini'
MODEL_GEMINI = genai.GenerativeModel("gemini-1.5-flash")



In [31]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
gemini_key = os.getenv('GEMINI_API_KEY')



if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    

API key looks good so far


In [32]:
# scraping patent
def get_patent_details(patnum):
    
    url = "https://patents.google.com/patent/" + patnum
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    title = soup.title.string if soup.title else "No title found"        
    
    full_text = soup.body.get_text(separator="\n", strip=True)
    
    first_clm= soup.find('div', attrs={'id':"CLM-00001"})
    if first_clm != None:
        first_claim = first_clm.text.strip()
    else:
        first_claim = "N/A"
    
    first_claim

    result = title + first_claim + full_text

    if result:
        return result
    else:
        return "patent details not found"


In [33]:
# system prompt

system_prompt = """You are a patent analyst who can understand patents and provide a summary that can be understood by a layman \
and also includes the technical aspects. The technical aspects are explained properly so that a layman can understand. \
Make sure the sentences are well-written, structured and connected to present information in a flow. Dont use long sentences.

Please provide the summary of the claim. There should be 2-3 paragraphs at maximum. In the first paragraph, focus the explanation on claims. Understand the scope of claim elements from the patent text provided below.
In the second paragraph, focus on the problem being solved by the patent and the advantages it brings.  

Here is an example of a patent summary:
The patent claims a vehicle which runs two operating systems simultaneously. These opearting systems run independently on a separate execution environments on shared kernel but \
the second operating system is not based on the shared kernel. The operating systems run separate application frameworks and the applications running on the second operating system \
are not compatible with the first operating system. This allows execution of applications generally not supported by vehicles with just one operating systems. THis allows access to a \
user's mobile applications on the vehicle display panel.

This summary was based on this claim and some excerpts from the patent (not provided below).
A vehicle, comprising:
a power source to propel the vehicle;
a display panel to provide graphical information to a vehicle occupant; and
an on board processing module, comprising:
a first operating system running in a first execution environment on a shared kernel, 
the first operating system having a first application framework supporting a first vehicle-related application; and
a second operating system running concurrently with the first operating system
in a second execution environment on the shared kernel, the second operating system having a second application framework supporting a second vehicle-related
application, wherein the second vehicle-related application is incompatible with the first application framework,

wherein the first operating system is based on the shared kernel and the second operating system is not based on the shared kernel
wherein the first and second operating systems run independently on the shared kernel, 
wherein at least one of the first and second vehicle-related applications is one of the following:
a vehicle control application, an application associated with at least one vehicle function, an application related to a handicap and
accessibility graphical user interface for the display panel, an email client, a web browser, a communications application, a game,
an entertainment application, a vehicle satellite positioning system receiver application, an automotive navigation application, a medical information application,
an emergency service application, a noise suppression application, a news-related application, 
a weather information-related application, a biometric application, a travel application, or a utility application.
"""


In [43]:
# user prompt
def get_user_prompt(patnum):
    patnumber = str(patnum)
    #print(patnumber)
    user_prompt = "Please provide summary of the patent. Here is patent number, title, first claim and full text of the patent:\n"
    user_prompt = user_prompt + patnum + "\n"
    user_prompt += get_patent_details(patnumber)
    return user_prompt

In [44]:
#request Gemini

def get_summary_gemini(patnum):
    print(f"Fetching summary for patent {patnum} from gemini.")
    genai.configure()
    gemini = genai.GenerativeModel(
        model_name='gemini-1.5-flash',
        system_instruction=system_prompt
    )
    gemini_response = gemini.generate_content(get_user_prompt(patnum), stream = True)
    
    results = ""
    for chunk in gemini_response:
        results += chunk.text or ""
        yield results

In [45]:
# request openAI

def get_summary_openai(patnum):
    print(f"Fetching summary for patent {patnum} from OpenAI.")
    openai = OpenAI()
    user_prompt = get_user_prompt(patnum)
    prompts = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
    ]

    openai_response = openai.chat.completions.create(model="gpt-4o-mini", messages=prompts, stream = True)

    results = ""
    for chunk in openai_response:
        results += chunk.choices[0].delta.content or ""
        #print(results)
        yield results

In [46]:
# Gradio 
with gr.Blocks() as ui:
    gr.Markdown("## Patent Summary Generator")

    with gr.Row():
        patent_input = gr.Textbox(label="Enter the Patent Number", placeholder="e.g., US12345678")

    with gr.Row():
        fetch_openai = gr.Button("Get OpenAI Summary")
        fetch_gemini = gr.Button("Get Gemini Summary")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### OpenAI Summary:")
            openai_summary = gr.Markdown()

        with gr.Column():
            gr.Markdown("### Gemini Summary:")
            gemini_summary = gr.Markdown()

    fetch_openai.click(get_summary_openai, inputs=[patent_input], outputs=[openai_summary])
    fetch_gemini.click(get_summary_gemini, inputs=[patent_input], outputs=[gemini_summary])

ui.launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


Fetching summary for patent US10000000 from OpenAI.
Fetching summary for patent US10000000 from gemini.
